In [1]:
from Fairness_aware_model import FairnessAwareModel
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.model_selection import train_test_split

In [2]:
dataCar = pd.read_csv('dataCar_clean.csv') # contient le train-test split pour comparer avec R au besoin
#clm = dataCar['clm']
#Xtrain, test, _, _ = train_test_split(dataCar, clm, test_size=0.2)
train=dataCar.loc[dataCar['train']==1]
test=dataCar.loc[dataCar['train']==0]

# Modèle baseline avec la méthode bfgs

In [3]:
mod = smf.glm(formula="clm ~ veh_value + veh_body + C(veh_age) + area + gender + C(agecat)", family=sm.families.Binomial(), data=train)
res = mod.fit(method="bfgs")
print(res.summary()) # donne la même chose qu'en R

                 Generalized Linear Model Regression Results                  
Dep. Variable:                    clm   No. Observations:                54285
Model:                            GLM   Df Residuals:                    54257
Model Family:                Binomial   Df Model:                           27
Link Function:                  Logit   Scale:                          1.0000
Method:                          bfgs   Log-Likelihood:                -13435.
Date:                Mon, 14 Nov 2022   Deviance:                       26870.
Time:                        08:38:50   Pearson chi2:                 5.43e+04
No. Iterations:                     0   Pseudo R-squ. (CS):           0.002612
Covariance Type:            nonrobust                                         
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            -1.9774      0.48

In [4]:
protected_attributes = ["gender", "agecat"]
# Standard scaling for regression
binary_answer = dataCar["clm"].values
numclaim = dataCar["numclaims"].values
reg_claim = dataCar["claimcst0"].values

In [5]:
"gender" in ["A", "b", "gender", "c"]

True

In [6]:
def hot_encoder(data, optional_columns):
    categorical_cols = []
    for column in data.columns:
            if data[column].dtype == object or column in optional_columns:
                categorical_cols.append(column)

    to_return = pd.get_dummies(data, columns = categorical_cols)
    return to_return

dataCar_encoded = hot_encoder(dataCar, ["gender", "agecat"])

dataCar_encoded = dataCar_encoded.drop(["clm", "numclaims", "claimcst0", "veh_body_BUS","area_A","exposure", "gender_F", "agecat_1"], axis=1)

In [7]:
train_encoded = dataCar_encoded.loc[dataCar_encoded['train']==1]
test_encoded = dataCar_encoded.loc[dataCar_encoded['train']==0]
y = dataCar['clm']

y_train = dataCar.loc[dataCar['train']==1, 'clm']
y_test = dataCar.loc[dataCar['train']==0, 'clm']
X_train_encoded = train_encoded.drop(["train"], axis=1)
X_test_encoded = test_encoded.drop(["train"], axis=1)

In [8]:
# question : à quoi ça sert de faire reshape?
beta = np.ones(X_train_encoded.shape[1])
predicted_x = X_train_encoded @ beta
print(np.sum(y_train.values.reshape(-1,1) * predicted_x.values.reshape(-1,1) - np.exp(predicted_x.values.reshape(-1,1))))
print(np.sum(y_train * predicted_x - np.exp(predicted_x)))

-5.5771202852141544e+16
-5.5771202852141544e+16


In [9]:
X_train_encoded.columns.values

array(['veh_value', 'veh_age', 'veh_body_CONVT', 'veh_body_COUPE',
       'veh_body_HBACK', 'veh_body_HDTOP', 'veh_body_MCARA',
       'veh_body_MIBUS', 'veh_body_PANVN', 'veh_body_RDSTR',
       'veh_body_SEDAN', 'veh_body_STNWG', 'veh_body_TRUCK',
       'veh_body_UTE', 'gender_M', 'area_B', 'area_C', 'area_D', 'area_E',
       'area_F', 'agecat_2', 'agecat_3', 'agecat_4', 'agecat_5',
       'agecat_6'], dtype=object)

In [10]:
model2 = sm.Logit(y_train, sm.add_constant(X_train_encoded)).fit()
print(model2.summary())
model2.params

Optimization terminated successfully.
         Current function value: 0.247637
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:                    clm   No. Observations:                54285
Model:                          Logit   Df Residuals:                    54259
Method:                           MLE   Df Model:                           25
Date:                Mon, 14 Nov 2022   Pseudo R-squ.:                0.004681
Time:                        08:38:51   Log-Likelihood:                -13443.
converged:                       True   LL-Null:                       -13506.
Covariance Type:            nonrobust   LLR p-value:                 1.596e-15
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
const             -1.8913      0.489     -3.865      0.000      -2.850      -0.932
veh_value        

const            -1.891278
veh_value         0.057412
veh_age           0.007808
veh_body_CONVT   -1.866944
veh_body_COUPE   -0.347482
veh_body_HBACK   -0.589844
veh_body_HDTOP   -0.434897
veh_body_MCARA    0.054627
veh_body_MIBUS   -0.648079
veh_body_PANVN   -0.350192
veh_body_RDSTR   -0.575863
veh_body_SEDAN   -0.562922
veh_body_STNWG   -0.526675
veh_body_TRUCK   -0.529253
veh_body_UTE     -0.780079
gender_M         -0.034863
area_B            0.099312
area_C            0.052815
area_D           -0.090240
area_E           -0.017174
area_F            0.070535
agecat_2         -0.224501
agecat_3         -0.273813
agecat_4         -0.294366
agecat_5         -0.485139
agecat_6         -0.541433
dtype: float64

In [11]:
protected_attributes = [np.where(dataCar.columns.values == i)[0][0] for i in protected_attributes] # position des colonnes d'attributs protégés dans les données

In [12]:
fam = FairnessAwareModel(regularization=0, protected_attributes=protected_attributes, family="binomial", offset=1)
fam.fit(np.column_stack([np.ones(X_train_encoded.shape[0]), np.array(X_train_encoded)]), y_train.values) # il faut inclute un "intercept", donc colonne de 1

Optimization terminated successfully.
         Current function value: 0.247637
         Iterations 8


/media/simon/Elements/Universite/Maitrise statistique/5-A22/Actulab-coop/Fairness_aware_model.py:66: RuntimeWarning: overflow encountered in exp
  return np.sum(y.reshape(-1,1) * lin_predictor - np.log(1+np.exp(lin_predictor)))
/home/simon/anaconda3/envs/actulab/lib/python3.9/site-packages/scipy/optimize/_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0


         Current function value: -16108.219337
         Iterations: 1
         Function evaluations: 3036
         Gradient evaluations: 112


In [13]:
pd.DataFrame({"variable":["intercept"] + X_train_encoded.columns.tolist(), "beta":fam.beta})

,variable,beta
0,intercept,-1.891278
1,veh_value,0.182412
2,veh_age,0.257808
3,veh_body_CONVT,-1.741944
4,veh_body_COUPE,-0.222482
5,veh_body_HBACK,-0.464844
6,veh_body_HDTOP,-0.434897
7,veh_body_MCARA,0.054627
8,veh_body_MIBUS,-0.523079
9,veh_body_PANVN,-0.350192
